In [5]:
import numpy as np
import logging

from geodesic import compute_shortest_paths_dijkstra_cugraph, compute_shortest_paths_dijkstra,construct_knn_graph, GPU_AVAILABLE
from dissimilarity import calculate_cs_dissimilarity_matrix
from dataset import npy_file_name_converter, read_npy_of_file, read_all_of_huaweicup

Round_idx = 1
File_idx = 3
bs_pos, tol_samp_num, anch_samp_num, port_num, ant_num, sc_num, anch_pos, H, d_geo = read_all_of_huaweicup(Round_idx,File_idx)
# np.set_printoptions(precision=5, threshold=10, edgeitems=10, linewidth=200)
# logging.info(f'最短路径矩阵的前 100 行 100 列：\n{d_geo[:100, :100]}')
# 初始化日志记录
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()


Processing Round 1 Case 3
Loading configuration data file
Loading input position file
Loading input CSI data of Case 3
Loading Channel CSI succeed
Channel GEO data not exist


In [6]:

# 计算所有点之间的CS距离,得到tol_samp_num*tol_samp_num大小的矩阵
# 文件路径和文件名设置
d_cs = read_npy_of_file(1, File_idx, 'DCS')

# 检查文件是否存在
if d_cs == None : 
    # 如果文件不存在，则计算数据并保存到文件
    adp_file = npy_file_name_converter(Round_idx,File_idx,'DCS')
    d_cs = calculate_cs_dissimilarity_matrix(H)
    np.save(adp_file, d_cs)
    print("计算并保存Dcs数据成功")

2024-07-20 16:56:19,307 - INFO - 开始计算CS距离


./CompetitionData1/Round1DCS3.npy does not exist
使用cuda进行计算


计算CS距离:   0%|          | 55/20000 [00:02<09:59, 33.24it/s]  

第 5 行计算后output矩阵的状态:
tensor([[0., 0., 0., 0.,  ..., 0., 0., 0., 0.],
        [0., 0., 0., 0.,  ..., 0., 0., 0., 0.],
        [0., 0., 0., 0.,  ..., 0., 0., 0., 0.],
        [0., 0., 0., 0.,  ..., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.,  ..., 0., 0., 0., 0.],
        [0., 0., 0., 0.,  ..., 0., 0., 0., 0.],
        [0., 0., 0., 0.,  ..., 0., 0., 0., 0.],
        [0., 0., 0., 0.,  ..., 0., 0., 0., 0.]], device='cuda:0')


计算CS距离: 100%|██████████| 20000/20000 [09:15<00:00, 36.02it/s]  
2024-07-20 17:05:35,765 - INFO - CS距离计算完毕，共计算了 400000000 对点之间的距离。


计算并保存Dcs数据成功


In [7]:

# 计算接地线距离
# 文件路径和文件名设置

if d_geo == None:
    # 如果文件不存在，则计算数据并保存到文件
    k = 20
    geo_file = npy_file_name_converter(Round_idx,File_idx,'GEO')
    np.set_printoptions(precision=5, threshold=10, edgeitems=10, linewidth=200)
    print(f'构建的相异度矩阵：\n{d_cs[:10, :10]}')  # 打印前 10 行 10 列用于检查
    
    # 构建 k-近邻图
    print('开始构建 k-近邻图...')
    knn_graph_matrix = construct_knn_graph(d_cs, k)
    
    # 选择使用 CuGraph 或 NetworkX 计算最短路径长度矩阵 D_geo
    if GPU_AVAILABLE:
        print('检测到 GPU，使用 CuGraph 计算最短路径...')
        D_geo = compute_shortest_paths_dijkstra_cugraph(knn_graph_matrix)
    else:
        print('未检测到 GPU，使用 NetworkX 计算最短路径...')
        D_geo = compute_shortest_paths_dijkstra(knn_graph_matrix)
    
    # 保存 D_geo
    np.save(geo_file, D_geo)
    print('最短路径矩阵已保存。')

构建的相异度矩阵：
[[1.08480e-05 8.08527e+02 8.10314e+02 8.11474e+02 8.15527e+02 6.33928e+02 8.15611e+02 8.13707e+02 8.15638e+02 8.14717e+02]
 [8.08527e+02 2.40803e-05 8.15691e+02 8.10610e+02 8.15381e+02 8.13366e+02 8.15723e+02 8.15929e+02 8.01281e+02 8.14880e+02]
 [8.10314e+02 8.15691e+02 1.01328e-05 6.12125e+02 8.15974e+02 8.11536e+02 8.15970e+02 8.15341e+02 8.15677e+02 8.15992e+02]
 [8.11474e+02 8.10610e+02 6.12125e+02 1.53780e-05 8.15972e+02 8.05992e+02 8.15934e+02 8.09195e+02 8.15931e+02 8.15934e+02]
 [8.15527e+02 8.15381e+02 8.15974e+02 8.15972e+02 1.19209e-06 8.15951e+02 8.15959e+02 8.15952e+02 8.14200e+02 7.94190e+02]
 [6.33928e+02 8.13366e+02 8.11536e+02 8.05992e+02 8.15951e+02 5.84126e-06 8.15619e+02 8.15976e+02 8.15246e+02 8.15677e+02]
 [8.15611e+02 8.15723e+02 8.15970e+02 8.15934e+02 8.15959e+02 8.15619e+02 3.93391e-06 8.15871e+02 6.42976e+02 8.15959e+02]
 [8.13707e+02 8.15929e+02 8.15341e+02 8.09195e+02 8.15952e+02 8.15976e+02 8.15871e+02 6.19888e-06 8.15871e+02 8.15418e+02]
 [8.15

2024-07-20 17:05:44,713 - INFO - 构建的 k-近邻图，这里对k进行排序，把最小的k个邻接矩阵搬到图里，并将对角线置为0：
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


检测到 GPU，使用 CuGraph 计算最短路径...


/root/miniconda3/lib/python3.10/site-packages/cugraph/structure/symmetrize.py:92: FutureWarning: Multi is deprecated and the removal of multi edges will no longer be supported from 'symmetrize'. Multi edges will be removed upon creation of graph instance.
  warnings.warn(
2024-07-20 17:05:46,417 - INFO - 开始计算最短路径...
计算最短路径节点数: 100%|██████████| 20000/20000 [09:23<00:00, 35.47it/s]
2024-07-20 17:15:10,336 - INFO - 计算完成，最短路径矩阵的前 10 行 10 列：
[[   0.      1340.87317 2298.31738 1864.95886 1723.66089  577.99213 2917.73022 1861.32861 2707.09888 1359.91138]
 [1340.87317    0.      2219.63257 2045.40125 1618.8269  1826.88208 2323.354   2257.46094 2119.87598 1984.53796]
 [2298.31763 2219.63257    0.       475.68597 1904.84729 1835.60413 2444.16357 1377.10754 2509.55127 2437.45752]
 [1864.95886 2045.40125  475.68597    0.      1909.62988 1402.24524 2710.16309  993.60449 2455.26074 2241.99463]
 [1723.66089 1618.8269  1904.84729 1909.62988    0.      1675.47363 2487.43945 2197.2627  2014.76807  999.0

最短路径矩阵已保存。
